In [1]:
import glob
import pandas as pd
import numpy as np
import environment2
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from collections import defaultdict

from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk
import warnings;   warnings.filterwarnings("ignore")

In [2]:
env = environment2.environment2()
user_list_2D = env.user_list_2D
user_list_3D = env.user_list_3D

In [3]:
user_list_2D[0]

'data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv'

In [4]:
fileNames2D=['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv']

In [5]:
# fileNames2D=user_list_2D

In [6]:
fileNames2D

['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv']

In [7]:
from collections import defaultdict
def get_probabilities(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()
    # print(dataframe.head(5))
    states=['Answering','Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=1


    current_state=None
    for i in range(len(dataframe)):
      if dataframe["State"][i]== current_state:
        dataframe["high-level-action"][i]="same"
      else:
        dataframe["high-level-action"][i]="change"
      current_state=dataframe["State"][i]

    for i in range(len(dataframe)):
      if dataframe['State'][i]=="Navigation":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

      elif dataframe['State'][i]=="Sensemaking":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])

      elif dataframe['State'][i]=="Foraging":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])
      else:
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Answeringsame'] + map['Answeringchange'])
      probabilities.append(probs)
    dataframe['probabilities']=probabilities
    return dataframe, map

In [13]:
from collections import defaultdict
def get_probabilities_subtask(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()
    # print(dataframe.head(5))
    states=['Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=1


    current_state=None

    for i in range(len(dataframe)):
      if dataframe['State'][i]=="Navigation":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

      elif dataframe['State'][i]=="Sensemaking":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])

      else :
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])

      probabilities.append(probs)
    return probabilities

In [8]:
from collections import defaultdict
def get_probabilities_region(dataframe):
    df=dataframe
    probabilities=[]
    map=defaultdict(lambda: defaultdict(int))
    # print(dataframe.head(5))
    actions=['Answering','Sensemaking','Foraging','Navigation']
    states=['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None']
    for state in states:
      for action in actions:
        map[state][action]=1

    for i in range(len(df)):
      sum = 0
      map[df['Most_frequent_region'][i]][df['State'][i]] +=1
      for act in map[df['Most_frequent_region'][i]]:
                sum += map[df['Most_frequent_region'][i]][act]
      probs=map[df['Most_frequent_region'][i]][df['State'][i]]   / sum
      probabilities.append(probs)
    df['probabilities']= probabilities
    return df, map

In [9]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None

    def ADF_Stationarity_Test(self, timeseries, printResults = True):
      #Dickey-Fuller test:
      adfTest = adfuller(timeseries, autolag='AIC')

      self.pValue = adfTest[1]

      if (self.pValue<self.SignificanceLevel):
          self.isStationary = True
      else:
          self.isStationary = False

      if printResults:
          dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
          #Add Critical Values
          for key,value in adfTest[4].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Augmented Dickey-Fuller Test Results:')
          print(dfResults)

    def Mann_Kendall_Stationarity_Test(self, timeseries, printResults = True):
      #Mann Kendall Trend Test:
      mktTest = mk.original_test(timeseries,self.SignificanceLevel)

      self.pValue = mktTest[2]
      self.trend= mktTest[0]
      self.isStationary = mktTest[1]

      if printResults:
          dfResults = pd.Series(mktTest[0:3], index=['Trend Type','Stationarity','P-Value'])
          #Add Critical Values
          for key,value in mktTest[3].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Mann Kendall Trend Test:')
          print(dfResults)

In [10]:
user=[]
annotated_subtask=[]
SouthEastAnswering=[]
SouthEastSensemaking=[]
SouthEastForaging=[]
SouthEastNavigation=[]
SouthAnswering=[]
SouthSensemaking=[]
SouthForaging=[]
SouthNavigation=[]
OhioValleyAnswering=[]
OhioValleySensemaking=[]
OhioValleyForaging=[]
OhioValleyNavigation=[]
NorthernRockiesPlainsAnswering=[]
NorthernRockiesPlainsSensemaking=[]
NorthernRockiesPlainsForaging=[]
NorthernRockiesPlainsNavigation=[]
SouthWestAnswering=[]
SouthWestSensemaking=[]
SouthWestForaging=[]
SouthWestNavigation=[]
NorthWestAnswering=[]
NorthWestSensemaking=[]
NorthWestForaging=[]
NorthWestNavigation=[]
UpperMidwestAnswering=[]
UpperMidwestSensemaking=[]
UpperMidwestForaging=[]
UpperMidwestNavigation=[]
OtherAnswering=[]
OtherSensemaking=[]
OtherForaging=[]
OtherNavigation=[]
NoneAnswering=[]
NoneSensemaking=[]
NoneForaging=[]
NoneNavigation=[]

output_dataframe=pd.DataFrame(columns=['User','annotated_subtask','SouthEastAnswering',
'SouthEastSensemaking',
'SouthEastForaging',
'SouthEastNavigation',
'SouthAnswering',
'SouthSensemaking',
'SouthForaging',
'SouthNavigation',
'OhioValleyAnswering',
'OhioValleySensemaking',
'OhioValleyForaging',
'OhioValleyNavigation',
'NorthernRockiesPlainsAnswering',
'NorthernRockiesPlainsSensemaking',
'NorthernRockiesPlainsForaging',
'NorthernRockiesPlainsNavigation',
'SouthWestAnswering',
'SouthWestSensemaking',
'SouthWestForaging',
'SouthWestNavigation',
'NorthWestAnswering',
'NorthWestSensemaking',
'NorthWestForaging',
'NorthWestNavigation',
'UpperMidwestAnswering',
'UpperMidwestSensemaking',
'UpperMidwestForaging',
'UpperMidwestNavigation',
'OtherAnswering',
'OtherSensemaking',
'OtherForaging',
'OtherNavigation',
'NoneAnswering',
'NoneSensemaking',
'NoneForaging',
'NoneNavigation'])


result_dataframe= pd.DataFrame(columns=['User','annotated_subtask','SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None'])

for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')


  # print(df.ROI_Cycle.unique())
  for k in  df['annotated_subtask'].unique():
    dfs,mappings=get_probabilities_region((df.loc[df['annotated_subtask'] == k]).reset_index(drop=True))
    mapper=dict()
    mapper['User']=u
    mapper['annotated_subtask']=k
    print("annotated_subtask: #######################", k)
    for j in ['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None']:
        #print("Region: ",j ,"Action Preferred: " ,max(mappings[j], key=mappings[j].get))
        maximum=max(mappings[j], key=mappings[j].get)
        if mappings[j][maximum] > 1 :
            action_preferred=maximum
        else:
            action_preferred='NA'
        mapper[j]=action_preferred
    some_df=pd.DataFrame([mapper])
    result_dataframe = result_dataframe.append(some_df, ignore_index=True)
    all_dfs.append(dfs)
    maps.append(mappings)
    annotated_subtask.append(k)
    user.append(u)

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv
annotated_subtask: ####################### Subtask_1
annotated_subtask: ####################### Subtask_2
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
annotated_subtask: ####################### Subtask_1
annotated_subtask: ####################### Subtask_2
annotated_subtask: ####################### Subtask_3
annotated_subtask: ####################### Subtask_4
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv
annotated_subtask: ####################### Subtask_1
annotated_subtask: ####################### Subtask_2
annotated_subtask: ####################### Subtask_3
annotated_subtask: ####################### Subtask_4
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv
annotated_subtask: #######

In [11]:

result_dataframe.to_csv("data/NDSI-2D\\region_new-actions_annotated_subtask.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)

In [12]:
result_dataframe.head(20)

,User,annotated_subtask,SouthEast,South,OhioValley,NorthernRockiesPlains,SouthWest,NorthWest,UpperMidwest,Northeast,West,Other,None
0,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,Subtask_1,NA,Foraging,NA,NA,NA,NA,Foraging,NA,NA,NA,Foraging
1,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,Subtask_2,NA,NA,NA,Sensemaking,NA,NA,Navigation,NA,NA,NA,NA
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,Subtask_1,NA,Foraging,NA,NA,NA,Navigation,NA,NA,Navigation,NA,Foraging
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,Subtask_2,NA,Navigation,NA,Navigation,NA,NA,Foraging,NA,Navigation,NA,Foraging
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,Subtask_3,NA,Foraging,NA,NA,NA,NA,NA,Navigation,NA,NA,Navigation
5,userid_44968286-f204-4ad6-a9b5-d95b38e97866,Subtask_4,NA,Foraging,NA,Navigation,Navigation,NA,Navigation,NA,NA,Sensemaking,Foraging
6,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,Subtask_1,NA,Foraging,NA,NA,NA,NA,NA,NA,NA,NA,Foraging
7,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,Subtask_2,NA,NA,NA,Sensemaking,NA,NA,NA,NA,Navigation,NA,Navigation
8,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,Subtask_3,NA,NA,NA,Navigation,NA,Navigation,NA,NA,Foraging,NA,Navigation
9,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,Subtask_4,NA,Foraging,NA,Navigation,NA,NA,Foraging,Foraging,Foraging,NA,Foraging


In [56]:
user=[]
annotated_subtask=[]
Foragingchange=[]
Foragingsame=[]
Foraging=[]
Sensemakingchange=[]
Sensemakingsame=[]
Sensemaking=[]
Navigationsame=[]
Navigationchange=[]
Navigation=[]


output_dataframe=pd.DataFrame(columns=['User','annotated_subtask','Foragingchange','Foragingsame','Sensemakingchange','Sensemakingsame','Navigationsame','Navigationchange','Foraging','Navigation','Sensemaking'])
for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')




  # print(df.ROI_Cycle.unique())
  for k in df['annotated_subtask'].unique():
    dfs,mappings=get_probabilities((df.loc[df['annotated_subtask'] == k]).reset_index(drop=True))
    print("annotated_subtask: #######################", k, " TimeSpent: ############## ", dfs['IdleTime'].sum(), len(dfs), round(dfs['IdleTime'].sum()/len(dfs),2) )
    for j in dfs.State.unique():
        series=(dfs.loc[(dfs['State'] == j) & (dfs["high-level-action"]=="same")])['probabilities']
        if len(series) > 2 :
            sTest_state = StationarityTests()
            sTest_state.Mann_Kendall_Stationarity_Test(series.values, printResults = False)
            print("State is: ", j, "  MKT Result is:  ", sTest_state.trend)
        else :
             print("State is: ", j, "  MKT Result is:  ","Not Enough Data")
    all_dfs.append(dfs)
    maps.append(mappings)
    annotated_subtask.append(k)
    user.append(u)
  for k in range(len(maps)):
      #print("Cycle: ",k, "Foragingchange: ",maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      Foragingchange.append(maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      #print("Cycle: ",k, "Foragingsame: ",maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      Foragingsame.append(maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']))
      Foraging.append((maps[k]['Foragingsame'] + maps[k]['Foragingchange']))


      #print("Cycle: ",k, "Navigationchange: ",maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      Navigationchange.append(maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      #print("Cycle: ",k, "Navigationsame: ",maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      Navigationsame.append(maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']))
      Navigation.append((maps[k]['Navigationsame'] + maps[k]['Navigationchange']))


      #print("Cycle: ",k, "Sensemakingchange: ",maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
      Sensemakingchange.append(maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
      #print("Cycle: ",k, "Sensemakingsame: ",maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
      Sensemakingsame.append(maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
      Sensemaking.append((maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))


      print("##################################################")


output_dataframe['User']=user
output_dataframe['annotated_subtask']=annotated_subtask
output_dataframe['Foragingchange']=Foragingchange
output_dataframe['Foragingsame']=Foragingsame
output_dataframe['Foraging']=Foraging

output_dataframe['Sensemakingchange']=Sensemakingchange
output_dataframe['Sensemakingsame']=Sensemakingsame
output_dataframe['Sensemaking']=Sensemaking

output_dataframe['Navigationsame']=Navigationsame
output_dataframe['Navigationchange']=Navigationchange
output_dataframe['Navigation']=Navigation

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv
annotated_subtask: ####################### Subtask_1  TimeSpent: ##############  89.844 14 6
State is:  Foraging   MKT Result is:   increasing
State is:  Navigation   MKT Result is:   Not Enough Data
annotated_subtask: ####################### Subtask_2  TimeSpent: ##############  86.023 23 4
State is:  Navigation   MKT Result is:   increasing
State is:  Sensemaking   MKT Result is:   increasing
##################################################
##################################################
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
annotated_subtask: ####################### Subtask_1  TimeSpent: ##############  110.288 25 4
State is:  Foraging   MKT Result is:   no trend
State is:  Navigation   MKT Result is:   no trend
State is:  Sensemaking   MKT Result is:   increasing
annotated_subtask: ##############

In [39]:
output_dataframe.head(10)

,User,annotated_subtask,Foragingchange,Foragingsame,Sensemakingchange,Sensemakingsame,Navigationsame,Navigationchange,Foraging,Navigation,Sensemaking
0,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,Subtask_1,0.363636,0.636364,0.500000,0.500000,0.428571,0.571429,11,7,2
1,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,Subtask_2,0.500000,0.500000,0.142857,0.857143,0.846154,0.153846,2,13,14
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,Subtask_1,0.428571,0.571429,0.300000,0.700000,0.642857,0.357143,7,14,10
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,Subtask_2,0.400000,0.600000,0.666667,0.333333,0.692308,0.307692,5,13,3
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,Subtask_3,0.500000,0.500000,0.666667,0.333333,0.750000,0.250000,4,12,3
5,userid_44968286-f204-4ad6-a9b5-d95b38e97866,Subtask_4,0.250000,0.750000,0.428571,0.571429,0.705882,0.294118,16,17,7
6,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,Subtask_1,0.250000,0.750000,0.500000,0.500000,0.500000,0.500000,8,2,2
7,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,Subtask_2,0.500000,0.500000,0.500000,0.500000,0.666667,0.333333,2,9,4
8,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,Subtask_3,0.500000,0.500000,0.428571,0.571429,0.700000,0.300000,6,20,7
9,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,Subtask_4,0.222222,0.777778,0.666667,0.333333,0.700000,0.300000,18,20,6


In [40]:
output_dataframe.to_csv("data/NDSI-2D\\probability_distribution_annotated_subtask.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)